# ANN CALSIM 3 style

This notebook creates the ANN structure as mentioned in 
```
Artificial Neural Network for Sacramento–San Joaquin Delta Flow–Salinity Relationship for CalSim 3.0
Nimal C. Jayasundara, M.ASCE1; Sanjaya A. Seneviratne2; Erik Reyes3; and Francis I. Chung
```

The input structure consists of 8 inputs and their antecedent conditions expressed in daily or moving averaged values

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras import layers
#import keras

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import panel as pn
import hvplot.pandas

In [ ]:
dfin_on = pd.read_csv('smscg_input_on.csv',index_col=0,parse_dates=True)
dfin_off = pd.read_csv('smscg_input_off.csv',index_col=0,parse_dates=True)

dfout_on = pd.read_csv('smscg_output_on.csv',index_col=0,parse_dates=True)
dfout_off = pd.read_csv('smscg_output_off.csv',index_col=0,parse_dates=True)

## Tensorflow Board Setup
A log directory to keep the training logs

Tensorboard starts a separate process and is best started from the command line. Open a command window and activate this environment (i.e. keras) and goto the current directory. Then type in
```
tensorboard --logdir=./tf_training_logs/ --port=6006
```

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir=./tf_training_logs/ --port=6006
import os
root_logdir = os.path.join(os.curdir, "tf_training_logs")
tensorboard_cb = keras.callbacks.TensorBoard(root_logdir)

# Calibration and Validation Periods
Calibration is from 1940 - 2015 and Validation from 1923 - 1939 as per the Calsim 3 ANN paper


In [ ]:
import annutils

output_locations=['CVP_INTAKE','MIDR_INTAKE','OLDR_CCF','ROLD024',
        'RSAC081','RSAC092','RSAN007','RSAN018','SLMZU003','SLMZU011','VICT_INTAKE']

output_location='RSAN018_EC'

In [ ]:
(xallc,yallc), (xallv,yallv), xscaler, yscaler = annutils.create_training_sets([dfin_on,dfin_off],[dfout_on[[output_location]],dfout_off[[output_location]]])
import joblib
joblib.dump((xscaler,yscaler),'%s-xyscaler.dump'%output_location)
# Define Sequential model with 3 layers
NFEATURES=126 # (8 + 10)*7
def build_model(nhidden1=8,nhidden2=2,act_func='sigmoid'):
    model = keras.Sequential(
        [
        layers.Input(shape=(NFEATURES)),
        layers.Dense(nhidden1, activation=act_func),
        layers.Dense(nhidden2, activation=act_func),
        layers.Dense(1, activation=keras.activations.linear)
        ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    #model.compile(optimizer=keras.optimizers.RMSprop(), loss="mse")
    return model
model=build_model(8,2,act_func='sigmoid')
display(model.summary())

history = model.fit(
    xallc,
    yallc,
    epochs=1000,
    batch_size=128,
    validation_data=(xallv,yallv),
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, mode="min",restore_best_weights=True),
        tensorboard_cb
    ],
)
model.save('%s_ff_8x2.h5'%output_location)

In [ ]:
pd.DataFrame(history.history).hvplot(logy=True)

In [ ]:
import annutils
print(output_location)

In [ ]:
annutils.show_performance(model,dfin_on,dfout_on[output_location],xscaler,yscaler)

In [ ]:
annutils.show_performance(model,dfin_off,dfout_off[output_location],xscaler,yscaler)